# Building symbolic metamodels

A symbolic metamodel takes as an input a machine learning model, and outputs a symbolic equation describing its response surface as illustrated in the Figure below. This notebook provides the steps needed for building a symbolic metamodel for an XGBoost model fitted to the "UCI absenteeism at work dataset: https://archive.ics.uci.edu/ml/datasets/Absenteeism+at+work

<img src="images/FigB1.png" width="640"/>

We first import the necessary modules for symbolic metamodeling

In [1]:
from pysymbolic.algorithms.symbolic_metamodeling import *

Next, we import the necessary libraries fordata processing, splitting the data into training and testing samples, and XGBoost model fitting 

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


data          = pd.read_csv("data/absenteeism.csv", delimiter=';')

feature_names = ['Transportation expense', 'Distance from Residence to Work',
                 'Service time', 'Age', 'Work load Average/day ', 'Hit target',
                 'Disciplinary failure', 'Education', 'Son', 'Social drinker',
                 'Social smoker', 'Pet', 'Weight', 'Height', 'Body mass index']

scaler        = MinMaxScaler(feature_range=(0, 1))
X             = scaler.fit_transform(data[feature_names])
Y             = ((data['Absenteeism time in hours'] > 4) * 1) 

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

model         = XGBClassifier()

model.fit(X_train, Y_train)

[09:38:04] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=16, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

Let us examine the AUC-ROC performance of the fitted XGBoost model on the test data

In [3]:
roc_auc_score(Y_test, model.predict_proba(X_test)[:, 1])

0.700890272148234

How much better does XGBoost perform compared to a standard logistic regression?

In [4]:
model_L = LogisticRegression()

model_L.fit(X_train, Y_train)

roc_auc_score(Y_test, model_L.predict_proba(X_test)[:, 1])

0.6709250144759699

Now let us create a symbolic metamodel instance. This takes the fitted model **model** and the training features **X_train** as follows

In [5]:
X_train.shape

(495, 15)

In [6]:
X_train.shape[0]

495

In [7]:
metamodel = symbolic_metamodel(model, X_train, 'regression')

metamodel.fit(num_iter=10, batch_size=X_train.shape[0], learning_rate=.01)

---- Tuning the basis functions ----


  0%|          | 0/15 [00:00<?, ?it/s]

----  Optimizing the metamodel  ----


  0%|          | 0/10 [00:00<?, ?it/s]

Now let us see how this metamodel performs using the **evaluate** method...

In [8]:
Y_metamodel = metamodel.evaluate(X_test)

roc_auc_score(Y_test, Y_metamodel)

0.6920599305153444

It performs close to the original XGBoost model! Now let us see the exact symbolic equation of the model in terms of Meijer-G functions

By invoking the **.fit()** method in the **symbolic_metamodel** class, we essentially transform the XGBoost model to a space of interpretable symbolic equations as shown in the Figure below, without much loss in predictive accuracy. 

<img src="images/FigB2.png" width="320"/>

Now we show how to extract the exact and approximate equation $g(x)$ from the metamodel class...

In [9]:
metamodel.exact_expression

1/(exp(0.170062204160917*re(X0**1.3139748398191*hyper((1.0, 1.0), (1.24144764692094,), 1.39031152275745*X0*exp_polar(I*pi))) - 0.233945747483995*re(X1**0.505499371838737*hyper((1.0, 1.0), (0.802627422970494,), 1.60976983229806*X1*exp_polar(I*pi))) + 0.227078958313112*re(X10**2.03401827307969*hyper((1.0, 1.0), (2.05238328850332,), 0.779379451855607*X10*exp_polar(I*pi))) + 3.0507219546345*re(X11**0.831919566583644*hyper((1.0, 1.0), (1.18577036985539,), 2.28959166344203*X11*exp_polar(I*pi))) - 0.993228375784589*re(X12**0.350568064061525*hyper((1.0, 1.0), (0.836179310212418,), 2.37872595845861*X12*exp_polar(I*pi))) + 0.344210506048721*re(X13**0.676323028415981*hyper((1.0, 1.0), (1.03738764655643,), 1.4252757433118*X13*exp_polar(I*pi))) - 1.22741114399205*re(X14**0.250318957217233*hyper((1.0, 1.0), (0.823158736976876,), 2.0689603828857*X14*exp_polar(I*pi))) - 1.00576735836884*re(X2**0.961913478535204*hyper((1.0, 1.0), (2.50778934274747,), 1.43823116414752*X2*exp_polar(I*pi))) - 0.2530647272

Because this equation involves Hypergeometric functions, we might prefer to work with the polynomial approximation...

In [10]:
metamodel.approx_expression

1/(0.428150208127628*exp(-0.0254854730921055*X0**3*X1**3 + 0.0162508238429374*X0**3*X10**3 + 0.013382640065814*X0**3*X11**3 + 0.123192423737779*X0**3*X12**3 + 0.0566014503560595*X0**3*X13**3 + 0.0342556878227181*X0**3*X14**3 - 0.0557491048494805*X0**3*X2**3 - 0.0193201946394565*X0**3*X3**3 + 0.0508449412324566*X0**3*X4**3 - 0.00244158414564849*X0**3*X5**3 - 0.0909553950932318*X0**3*X6**3 + 0.0390487717942319*X0**3*X7**3 - 0.0154948429381363*X0**3*X8**3 + 0.0151238074836336*X0**3*X9**3 - 0.000515890743463783*X0**3 + 0.0108409578661944*X0**2*X1**2 - 0.00727467599791532*X0**2*X10**2 - 0.0059006383090203*X0**2*X11**2 - 0.0551826932720303*X0**2*X12**2 - 0.0252755362420184*X0**2*X13**2 - 0.0153686665480249*X0**2*X14**2 + 0.0250952077509866*X0**2*X2**2 + 0.00879940067465277*X0**2*X3**2 - 0.022542880828326*X0**2*X4**2 + 0.0010958443373111*X0**2*X5**2 + 0.0412311876181684*X0**2*X6**2 - 0.0173904138151816*X0**2*X7**2 + 0.00687237873423053*X0**2*X8**2 - 0.00638689477814746*X0**2*X9**2 - 0.0188365